# Travail pratique N°5 sur la mise en place d'une étude comparative en apprentissage

## réalisé par MULAPI TITA Ketsia

In [2]:
import time

In [3]:
import os

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.pipeline import make_pipeline
from sklearn.svm import LinearSVC, SVC
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, auc
from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning
import warnings
warnings.simplefilter("ignore", UserWarning)

In [5]:
def load_CSV_dataset(name):
    '''
    -- Cette méthode charge les données.
    - elle attend en paramètre le nom d'un fichier .csv qui représente un jeu de donnée et,
    - renvoie un dataframe.
    '''
    # TODO
    datas = pd.read_csv(os.path.join("datasets/", name)) #path,filename
    c = "Class" if name == "vehicle.csv" else "class" 
    return LabelEncoder().fit_transform(datas[c]), datas.drop(c, axis=1).to_numpy()

In [6]:
def split_dataset(X, y):
    '''
    -- Cette fonction fait du prétraitement :
    - elle standardise les données (en les ramenant à l'origine et en ramenant à la même échelle)
    - elle les sépare en données d'entraînement et en donnée de test
    - elle renvoie X_train, X_test, y_train, y_test
    '''
    # TODO
    
    X = StandardScaler().fit_transform(X)

    ssk = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
    ssk.get_n_splits(X, y)
    
    for train_index, test_index in ssk.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
    
    return ssk, X_train, y_train, X_test, y_test

In [41]:
@ignore_warnings(category=ConvergenceWarning)
def run_logreg(data):
    '''
    --Cette fonction permet de faire de la classification 
    - en utilisant une Regression_Logistique.
    - On chercher les hyper paramètres
    - on prédit et, on calcul un score de performance moyen
    '''
    
    # TODO
    
    ssk, X_train, y_train, X_test, y_test = data
    
    tuned_parameters = [
    {
     'penalty' : ['l1', 'l2', 'none'],
    'C' : np.logspace(-4, 4, 20),
    'solver' : ['liblinear']
    }
    ]

    model = GridSearchCV(LogisticRegression(), tuned_parameters, cv = ssk, verbose=True, n_jobs=-1)
    model.fit(X_train,y_train)
        
    c, p, s = model.best_params_['C'], model.best_params_['penalty'], model.best_params_['solver']
    
    clf = LogisticRegression(solver = s,penalty = p, C=c)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    
    score_accu_moyen = np.mean(cross_val_score(clf, X_test, y_test, cv=5))
    
    print("\n - log_reg : ", model.best_params_ ," -> ",model.best_score_, " & accuracy_score : ",score_accu_moyen)
    
    return score_accu_moyen

In [42]:
@ignore_warnings(category=ConvergenceWarning)
def run_linsvm(data):
    '''
    --Cette fonction permet de faire de la classification 
    - en utilisant un SVM Linéaire.
    - On chercher les hyper paramètres
    - on prédit et, on calcul un score de performance moyen
    '''
    # TODO
    
    ssk, X_train, y_train, X_test, y_test = data
    
    tuned_parameters = [{'C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]}]
    
    model = GridSearchCV(LinearSVC(), tuned_parameters, cv=ssk, n_jobs=5)
    model.fit(X_train,y_train)
    
    c = model.best_params_['C']
    
    clf = LinearSVC(C=c)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    
    score_accu_moyen = np.mean(cross_val_score(clf, X_test, y_test, cv=5))
    
    print("\n - linsvm : ", model.best_params_ ," -> ",model.best_score_, " & accuracy_score : ",score_accu_moyen)
    
    return score_accu_moyen

In [43]:
@ignore_warnings(category=ConvergenceWarning)
def run_rbfsvm(data):
    '''
    --Cette fonction permet de faire de la classification 
    - en utilisant une SVM à noyau rbf.
    - On chercher les hyper paramètres
    - on prédit et, on calcul un score de performance moyen
    '''
    
    # TODO
    ssk, X_train, y_train, X_test, y_test = data
    
    tuned_parameters = [{'gamma': [1e-3, 1e-4, 'scale', 'auto'],
                     'C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]}]
    
    model = GridSearchCV(SVC(kernel='rbf', random_state=42, break_ties=True), tuned_parameters, cv=ssk, n_jobs=5)
    model.fit(X_train,y_train)
    
    c, g = model.best_params_['C'], model.best_params_['gamma']
    
    clf = SVC(kernel = 'rbf', C=c, gamma = g)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    
    score_accu_moyen = np.mean(cross_val_score(clf, X_test, y_test, cv=5))
    
    print("\n - rbfsvm : ", model.best_params_ ," -> ",model.best_score_, " & accuracy_score : ",score_accu_moyen)
    
    return score_accu_moyen

In [38]:
def process_results(res_logreg , res_linsvm , res_rbfsvm):
    
    '''
    Cette focntion affiche les scores dans un dataframe, et présente les meilleurs scores
    '''
    
    # TODO
    
    datasets = ["heart", "diabetes", "vehicle", "segment", "spambase"]
    
    print("\n")
    
    df = pd.DataFrame({'datasets':datasets, 'Regression logistique':res_logreg,'SVM Linéaire':res_linsvm,'SVM avec noyau RBF':res_rbfsvm})
    df.style.background_gradient(cmap="Blues")
    
    for d in range(len(datasets)):        
        model = [res_logreg[d], res_linsvm[d], res_rbfsvm[d]]
        print("- Pour le jeux ", datasets[d], " le meilleur résultat est ", model[model.index(max(model))])
        
    return df

In [44]:
if __name__ == '__main__':
    ds_names = ["heart", "diabetes", "vehicle", "segment", "spambase"]
    res_logreg = []
    res_linsvm = []
    res_rbfsvm = []
    for name in ds_names:
        start = time.time()
        name += ".csv" 
        print('\n ******',name,'******')
        y, X = load_CSV_dataset(name)
        data = split_dataset(X, y)
        res_logreg.append(run_logreg(data))
        res_linsvm.append(run_linsvm(data))
        res_rbfsvm.append(run_rbfsvm(data))
        end = time.time()
        print("temps : ", end - start," ns")
        
    df = process_results(res_logreg , res_linsvm , res_rbfsvm)
    
    df


 ****** heart.csv ******
Fitting 5 folds for each of 60 candidates, totalling 300 fits

 - log_reg :  {'C': 0.03359818286283781, 'penalty': 'l2', 'solver': 'liblinear'}  ->  0.8426004228329811  & accuracy_score :  0.8345454545454546

 - linsvm :  {'C': 0.1}  ->  0.8380549682875265  & accuracy_score :  0.8145454545454545

 - rbfsvm :  {'C': 100, 'gamma': 0.001}  ->  0.856553911205074  & accuracy_score :  0.8163636363636364
temps :  5.895846128463745  ns

 ****** diabetes.csv ******
Fitting 5 folds for each of 60 candidates, totalling 300 fits

 - log_reg :  {'C': 0.23357214690901212, 'penalty': 'l1', 'solver': 'liblinear'}  ->  0.7772357723577235  & accuracy_score :  0.7584946236559139

 - linsvm :  {'C': 10}  ->  0.775609756097561  & accuracy_score :  0.7647311827956988

 - rbfsvm :  {'C': 1000, 'gamma': 0.001}  ->  0.767479674796748  & accuracy_score :  0.7382795698924731
temps :  6.132757902145386  ns

 ****** vehicle.csv ******
Fitting 5 folds for each of 60 candidates, totalling 3

In [40]:
df

datasets  Regression logistique  SVM Linéaire  SVM avec noyau RBF
0     heart               0.834545      0.814545            0.816364
1  diabetes               0.758495      0.771398            0.738280
2   vehicle               0.763636      0.781462            0.769697
3   segment               0.926438      0.915591            0.928565
4  spambase               0.895652      0.882609            0.907609

## Conclusion

- Temps d'éxécution: selon le temps d'éxécution, nous avons classons les jeux de données en fonction du temps de traitment 

     - heart (4.29 ns)
     - diabetes (5.80 ns)
     - vehicles (31.58 ns)
     - spambase (144.74 ns)
     - segment (200.58 ns)
     
       on peut déduire que plus un jeux de données est grand plus il est complexe à manipuler et donc, qu'il nécessitera plus de temps de traitement.

- Regression logistique : à l'instar du temps d'éxécution, on peut affirmer que les problèmes les plus couteux nécéssite un hyper paramètres à valeur élevés contrairement au SVM linéaire pour le quel C semble moins stricte, et pourtant en observant le SVM rbf, on observe une variation et une combinaison entre C et gamma qu'on ne peut pas visuellement lier à la taille du problème.

- Model Selection : Enfin, selon les scores moyens obtnus à partir de plusieurs validation croisée, il en ressort que pour chaque jeu, il existe un modèle qui  soit meilleur tel que :

     - heart (Regression Logistique)
     - diabetes (SVM Linéaire)
     - vehicles (SVM Linéaire)
     - segment (SVM avec noyau RBF)
     - spambase (SVM avec noyau RBF)
     
Les SVM à noyau donnant de bon résultat me pousse donc à conclure que l'approche kernel sépare mieux les données.